In [1]:
# Install dependencies
!pip install diffusers transformers accelerate opencv-python numpy imageio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
from diffusers import StableDiffusionPipeline
import numpy as np
import cv2
import imageio
from tqdm import tqdm

# Load Stable Diffusion model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to(device)

# Function to generate images
def generate_image(prompt, seed=None):
    generator = torch.manual_seed(seed) if seed else None
    image = pipe(prompt, num_inference_steps=30, generator=generator).images[0]
    return np.array(image)

# Take user input for prompts
prompts = []
num_prompts = int(input("Enter the number of prompts you want to provide: "))
for i in range(num_prompts):
    prompt = input(f"Enter prompt {i+1}: ")
    prompts.append(prompt)

# Generate frames
frames = []
for i, prompt in enumerate(prompts):
    print(f"Generating frame {i+1}/{len(prompts)}...")
    img = generate_image(prompt, seed=42 + i)
    frames.append(img)

# Interpolate frames using OpenCV
def interpolate_frames(frames, num_interpolated=10):
    new_frames = []
    for i in range(len(frames) - 1):
        img1, img2 = frames[i], frames[i + 1]
        for alpha in np.linspace(0, 1, num_interpolated):
            blended = cv2.addWeighted(img1, 1 - alpha, img2, alpha, 0)
            new_frames.append(blended)
    new_frames.append(frames[-1])
    return new_frames

# Smooth interpolation
video_frames = interpolate_frames(frames, num_interpolated=20)

# Save as video
video_path = "ai_animation.mp4"
imageio.mimsave(video_path, video_frames, fps=10)

print(f"Video saved as {video_path}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Enter the number of prompts you want to provide: 3
Enter prompt 1: A futuristic city at sunset.
Enter prompt 2: A futuristic city at night with neon lights.
Enter prompt 3: A futuristic city with flying cars in the morning.
Generating frame 1/3...


  0%|          | 0/30 [00:00<?, ?it/s]

Generating frame 2/3...


  0%|          | 0/30 [00:00<?, ?it/s]

Generating frame 3/3...


  0%|          | 0/30 [00:00<?, ?it/s]

Video saved as ai_animation.mp4
